<a href="https://colab.research.google.com/github/Bambika69/DeepReinforcementLearningHuggingface/blob/main/A2C_AntBulletEnv_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

In [ ]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [ ]:
# Install the specific setuptools version required to install the dependencies
!pip install setuptools==65.5.0

In [ ]:
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit6/requirements-unit6.txt

In [ ]:
import pybullet_envs
import panda_gym
import gym

import os

from huggingface_sb3 import load_from_hub, package_to_hub

from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env

from huggingface_hub import notebook_login

In [ ]:
env_id = "AntBulletEnv-v0"
# Create the env
env = gym.make(env_id)

# Get the state space and action space
s_size = env.observation_space.shape[0]
a_size = env.action_space

https://hackmd.io/@jeffreymo/SJJrSJh5_#PyBullet

In [ ]:
print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample())  # Get a random observation

_____OBSERVATION SPACE_____ 

The State Space is:  28
Sample observation [ 1.090922    0.0366804   0.20170271 -0.7913821  -0.05507981  1.9332827
  2.1874554   0.30170634 -0.21550559  1.043578    1.1461115   1.1831931
 -0.33332095  1.1151187   0.25819802 -0.02555911 -0.25139725 -0.54232997
 -1.3458718   2.0041964   0.20413901  1.7878771   0.10816936 -1.253113
  0.50124234 -2.2672668   2.1540768  -0.6549401 ]


In [ ]:
print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample())  # Take a random action


 _____ACTION SPACE_____ 

The Action Space is:  Box([-1. -1. -1. -1. -1. -1. -1. -1.], [1. 1. 1. 1. 1. 1. 1. 1.], (8,), float32)
Action Space Sample [-0.25371164 -0.29642043 -0.83026266  0.2766906   0.89662594 -0.6145385
  0.16148868 -0.15438491]


RL Tips & Tricks

https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html

In [ ]:
# Normalize observation and rewards
env = make_vec_env(env_id, n_envs=4)

env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.0)

A2C implementation with StableBaselines3

https://stable-baselines3.readthedocs.io/en/master/modules/a2c.html#notes

To find the best parameters:

https://huggingface.co/sb3

In [ ]:
model = A2C(
    policy="MlpPolicy",
    env=env,
    gae_lambda=0.9,
    gamma=0.99,
    learning_rate=0.00096,
    max_grad_norm=0.5,
    n_steps=8,
    vf_coef=0.4,
    ent_coef=0.0,
    policy_kwargs=dict(log_std_init=-2, ortho_init=False),
    normalize_advantage=False,
    use_rms_prop=True,
    use_sde=True,
    verbose=1,
)

Using cuda device


In [ ]:
model.learn(2_000_000)

In [ ]:
# Save the model and  VecNormalize statistics when saving the agent
model.save("a2c-AntBulletEnv-v0")
env.save("vec_normalize.pkl")

In [ ]:
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# Load the saved statistics
eval_env = DummyVecEnv([lambda: gym.make("AntBulletEnv-v0")])
eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)

#  do not update them at test time
eval_env.training = False
# reward normalization is not needed at test time
eval_env.norm_reward = False

# Load the agent
model = A2C.load("a2c-AntBulletEnv-v0")

mean_reward, std_reward = evaluate_policy(model, env)

print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

Mean reward = 1877.70 +/- 95.22


In [ ]:
notebook_login()
!git config --global credential.helper store

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
package_to_hub(
    model=model,
    model_name=f"a2c-{env_id}",
    model_architecture="A2C",
    env_id=env_id,
    eval_env=eval_env,
    repo_id=f"Bandika/a2c-{env_id}",
    commit_message="Initial commit",
)

ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.


/usr/local/lib/python3.9/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Saving video to /tmp/tmpm9yhzts2/-step-0-to-step-1000.mp4
ℹ Pushing repo Bandika/a2c-AntBulletEnv-v0 to the Hugging Face Hub


policy.pth:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

...

policy.optimizer.pth:   0%|          | 0.00/56.2k [00:00<?, ?B/s]

replay.mp4:   0%|          | 0.00/1.09M [00:00<?, ?B/s]

vec_normalize.pkl:   0%|          | 0.00/2.49k [00:00<?, ?B/s]

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/Bandika/a2c-AntBulletEnv-v0/tree/main/


'https://huggingface.co/Bandika/a2c-AntBulletEnv-v0/tree/main/'